<a href="https://colab.research.google.com/github/vhcarrillo/CPY16S/blob/main/CPY16S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is for the analysis of 16S amplicon sequencing form cenotes in the Yucatan Peninsula

Fist, import the dataset from a repo in Github

In [5]:
!git clone https://github.com/vhcarrillo/CPY16S.git

Cloning into 'CPY16S'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), done.


The original files listed in the repo are:

In [8]:
%ls CPY16S/

metadata.tsv       rep-seqs-cenotes.qza    table-freq-cenotes.qzv
paper_sep18.ipynb  setup_qiime2.py
README.md          table-freq-cenotes.qza


*   metadata.tsv that contains the metadata for the sites of study
*   



In the repository is there a file to install qiime2 named setup_qiime2.py got it from https://github.com/Gibbons-Lab/isb_course_2021/blob/main/setup_qiime2

In [6]:
%run CPY16S/setup_qiime2.py

[18:36:00] 🐍 Downloading miniconda...                                     ]8;id=318099;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=166903;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

[18:36:01] 🐍 Done.                                                        ]8;id=899596;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=972318;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           🐍 Installing miniconda...                                      ]8;id=37530;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=661258;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

[18:36:30] 🐍 Installed miniconda to `/usr/local` 🐍                       ]8;id=535932;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=319907;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           🔍 Downloading Qiime 2 package list...                          ]8;id=968124;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=441922;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

...

[18:44:28] 📊 Qiime 2 command line looks good 🎉                           ]8;id=462577;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=245740;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           📊 Checking if Qiime 2 import works...                         ]8;id=570715;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=68494;file:///content/CPY16S/setup_qiime2.py#103\103]8;;\

[18:44:29] 📊 Qiime 2 can be imported 🎉                                  ]8;id=972673;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=507564;file:///content/CPY16S/setup_qiime2.py#109\109]8;;\

           Cleaned up unneeded files.                                      ]8;id=650407;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=851153;file:///content/CPY16S/setup_qiime2.py#34\34]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍         ]8;id=347839;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=731855;file:///content/CPY16S/setup_qiime2.py#113\113]8;;\